<h4 style="font-variant-caps: small-caps;font-size:35pt;">Databricks-ML-professional-S01a-Data-management</h4>

<div style='background-color:black;border-radius:5px;border-top:1px solid'></div>
<br/>
<p>This Notebook adds information related to the following requirements:</p><br/>
<b>Data Management:</b>
<ul>
<li>Read and write a Delta table</li>
<li>View Delta table history and load a previous version of a Delta table</li>
<li>Create, overwrite, merge, and read Feature Store tables in machine learning workflows</li>
</ul>
<br/>
<p><b>Download this notebook at format ipynb <a href="Databricks-ML-professional-S01a-Data-management.ipynb">here</a>.</b></p>
<br/>
<div style='background-color:black;border-radius:5px;border-top:1px solid'></div>

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">1. Import libraries</span></div>

In [0]:
import pandas as pd
import seaborn as sns
#
from pyspark.sql.functions import *
#
from databricks.feature_store import FeatureStoreClient, feature_table

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">2. Load dataset, convert to Spark DataFrame</span></div>

In [0]:
taxis_df = sns.load_dataset("taxis")
#
taxis_sdf = spark.createDataFrame(taxis_df)
#
display(taxis_sdf.limit(5))

<a id="readwritedeltatable"></a>
<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">3. Read and write a Delta table</span></div>

<p>Dataframes are saved by default as <b>managed delta tables</b> when the location is not specified:</p>

In [0]:
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .saveAsTable("taxis_sdf"))

<p>When location is specified, they are <b>external delta tables</b>:</p>

In [0]:
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .save("/temp/taxis_sdf"))

<p><b>Managed delta tables</b> can be listed with the following command:</p>

In [0]:
display(spark.sql("show tables").limit(5))

<p>Delta tables can be read this way:</p>

In [0]:
taxis_sdf    = spark.table("taxis_sdf")           # read managed delta table
taxi_sdf_ext = spark.read.load("/temp/taxis_sdf") # read external delta table

In [0]:
display(taxi_sdf_ext.limit(5))

<a id="viewdeltahistory"></a>
<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">4. View Delta table history and load a previous version of a Delta table</span></div>

<p>Let's do a small modification to the table and save it again:</p>

In [0]:
# remove nulls for payment column
taxis_sdf = taxis_sdf.filter("payment is not null")
#
# save the change to managed delta table
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .saveAsTable("taxis_sdf"))
#
# save the change to external delta table
(taxis_sdf.write
          .mode("overwrite")
          .option("overwriteSchema", "True")
          .format("delta")
          .save("/temp/taxis_sdf"))

<p>See changes in <b>managed table</b>:</p>

In [0]:
spark.sql("DESCRIBE HISTORY taxis_sdf")

<p>See changes in <b>external table</b>:</p>

In [0]:
spark.sql("DESCRIBE HISTORY '/temp/taxis_sdf/'")

<p>The following command lets load a previous version. We see that loading version <code>0</code> of table get null values for column <code>payment</code> back:</p>

In [0]:
display(spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .table("taxis_sdf")
        .filter("payment is null")
        .limit(5))
#
# Or for an external delta table
display(spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load("/temp/taxis_sdf")
        .filter("payment is null")
        .limit(5))

<a id="actionsfeaturestoretables"></a>
<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">5. Create, overwrite, merge and read Feature Store tables in machine learning workflows</span></div>

<p>Let's add an index to the table:</p>

In [0]:
taxis_sdf = taxis_sdf.withColumn("id", monotonically_increasing_id())

<p>A Catalog and schema must exist. Then if one them is missing, it is necessary to create it/them. For the example here, let's use the default existing Catalog <code>hive_metastore</code> and create a Schema in it:</p>

In [0]:
# Create a catalog
spark.sql("USE CATALOG hive_metastore")
spark.sql("CREATE SCHEMA IF NOT EXISTS seaborn_db")

<p>The following command is to create and populate a feature store table in schema <code>seaborn_df</code> from <code>hive_metastore</code> Catalog:</p>

In [0]:
fs = FeatureStoreClient()
#
customer_feature_table = fs.create_table(
  name='hive_metastore.seaborn_db.taxidataset', # format is <catalog_name>.<schema_name>.<table_name>
  primary_keys='id',                            # required
  schema=taxis_sdf.schema,                      # either schema or df parameter is required - if df is provided, the feature store table will be automatically populated with the data, otherwise only the structure of the table will be created, it will need to be populated later
  df = taxis_sdf,                               # either schema or df parameter is required - if df is provided, the feature store table will be automatically populated with the data, otherwise only the structure of the table will be created, it will need to be populated later
  description='Seaborn taxi dataset features'
)

<p>The previously created table in Feature Store can be deleted this way:</p>

In [0]:
fs.drop_table(name='seaborn_db.taxidataset')

<p>Alternatively, an empty table structure can be created in the Feature store to be populated later. The command below only creates the structure of the table in Feature Store:</p>

In [0]:
customer_feature_table = fs.create_table(
  name='hive_metastore.seaborn_db.taxidataset', # format is <catalog_name>.<schema_name>.<table_name>
  primary_keys='id',                            # required
  schema=taxis_sdf.schema,                      # here only schema is provided, the feature table is created empty
  description='Seaborn taxi dataset features'
)

<p>Next step is to write data in it:</p>

In [0]:
fs.write_table(
  df=taxis_sdf,
  name='seaborn_db.taxidataset',
  mode='merge'                   # mode = 'overwrite' could also be used in this particular case
)

<p>Finally, feature store table can be read this way:</p>

In [0]:
display(fs.read_table(name='seaborn_db.taxidataset').limit(5))

<img width="1000px" src="https://i.ibb.co/Yy56SQz/unity-catalog-1.png"/>